In [ ]:
##LANDSAT DATA/NDVI DATA -- PLUS CBGs

import ee 
import folium
import geehydro
import json
import requests
from datetime import datetime as dt
from IPython.display import Image

# initialize the connection to the server
ee.Initialize()

maricopa = [[-113.6318726643051,32.3815643902006],
            [-110.8797974689926,32.3815643902006],
            [-110.8797974689926,34.413007528786],
            [-113.6318726643051,34.413007528786],
            [-113.6318726643051,32.3815643902006]] 
  
maricopa_AOI = ee.Geometry.Polygon(maricopa)


# polygon for where phoenix  is
poly = [[-112.35608531078948,33.22139301900224], 
        [-111.89740611157073,33.22139301900224],  
        [-111.89740611157073,33.93760032210363], 
        [-112.35608531078948,33.93760032210363], 
        [-112.35608531078948,33.22139301900224]] 

phoenix_AOI = ee.Geometry.Polygon(poly)

#pulling LANDSAT data from google 
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
landsat_AOI = landsat.filterBounds(phoenix_AOI).filterDate('2020-06-21', '2020-9-22').sort('CLOUD_COVER')

#taking the median of data from the collection range; multiple satellite images 
phoenix = landsat_AOI.median().clip(maricopa_AOI)

parameters = {'min': 0,
              'max': 3000,
              'scale':30,
              'bands': ['B1', 'B2', 'B3'],
              'gamma': 1.4,
              'crs': 'EPSG:3857',
              'maxPixels': 10e13,
              'region': phoenix_AOI}

#calculating NDVI
ndvi = phoenix.normalizedDifference(['B5', 'B4'])

palette = ['red', 'yellow', 'green']
ndvi_parameters = {'min': 0,
                   'max': 1,
                   'scale':30,
                   'palette': palette,
                   'crs': 'EPSG:3857',
                   'region': phoenix_AOI}

##ADDING CBG DATA
#creating jsons for CBG, and a filter CBG that we pass back into the CBG
CBG = json.load(open("Census_Block_Group_(TIGER_Line)_-_Arizona_(2010)_.geojson"))
Duplicate_CBG = json.load(open("Census_Block_Group_(TIGER_Line)_-_Arizona_(2010)_.geojson"))

#trimming values so we can add to map
for i in range(len(Duplicate_CBG['features'])):
    value = Duplicate_CBG['features'][i]['properties']['STATEFP10'] + Duplicate_CBG['features'][i]['properties']['COUNTYFP10'] + Duplicate_CBG['features'][i]['properties']['TRACTCE10']
    CBG['features'][i]['properties']['GEOID10'] = value
    
tract_array = []  
phoenix_CBGs = []

#turning the valid tracts into an index we search for if the tracts are in maricopa
tracts = open("TRACTS.txt")
for line in tracts:
    key = line.split(";")[1]
    tract_array.append(key)

#checking to see if the CBG is in maricopa, if it is, we add it to our list of CBGs to include in map
for i in range(len(CBG['features'])):
    for j in range(len(tract_array)):
        if tract_array[j].strip() == CBG['features'][i]['properties']['GEOID10']:
            phoenix_CBGs.append(CBG['features'][i])
            
#creating a json file from our CBGs in maricopa            
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = phoenix_CBGs

##ADDING HEAT

# polygon for where maricopa county is

heat_poly = [[-113.2724447981062,32.61418955304137],
             [-110.2621908918562,32.61418955304137],
             [-110.2621908918562,34.50002029652324],
             [-113.2724447981062,34.50002029652324],
             [-113.2724447981062,32.61418955304137]] 



heat_AOI = ee.Geometry.Polygon(heat_poly)

MODIS = ee.ImageCollection("MODIS/006/MOD11A1")
MODIS_AOI = MODIS.filterDate('2020-1-21', '2020-3-22').select('LST_Day_1km')

phoenix_heat = MODIS_AOI.mean().clip(heat_AOI)


heat_parameters = {'min': 13000,
              'max': 16750,
              'palette': [
                '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
                '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
                '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
                'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
                'ff0000', 'de0101', 'c21301', 'a71001', '911003'
                          ],}
    
phoenix_map = folium.Map(location=[33.4484, -112.0740], zoom_start=7)


#creating map
phoenix_map = folium.Map(location=[33.4484, -112.0740], zoom_start=7)
phoenix_map.addLayer(ndvi, ndvi_parameters, name = "NDVI")
folium.GeoJson(new_json, name = "CBGs").add_to(phoenix_map)
phoenix_map.addLayer(phoenix_heat, heat_parameters, name = "HEAT")
folium.LayerControl().add_to(phoenix_map)

phoenix_map


In [12]:
#CHUNK OF CODE FOR THE LAND SURFACE TEMP CALC -- INCLUDES NDVI

import ee 
import folium
import geehydro
import json
import requests
from datetime import datetime as dt
from IPython.display import Image

ee.Initialize()

maricopa = [[-113.6318726643051,32.3815643902006],
            [-110.8797974689926,32.3815643902006],
            [-110.8797974689926,34.413007528786],
            [-113.6318726643051,34.413007528786]] 
  
maricopa_AOI = ee.Geometry.Polygon(maricopa)

phoenix_point = ee.Geometry.Point([-112.0740,33.4484])


# polygon for where phoenix  is
poly = [[-112.35608531078948,33.22139301900224], 
        [-111.89740611157073,33.22139301900224],  
        [-111.89740611157073,33.93760032210363], 
        [-112.35608531078948,33.93760032210363]] 

phoenix_AOI = ee.Geometry.Polygon(poly)
maricopa_AOI = ee.Geometry.Polygon(maricopa)

landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterDate('2020-06-21','2021-09-22')
landsat_AOI = landsat.filterBounds(phoenix_point).filterMetadata('CLOUD_COVER', 'less_than', 0.01)


print('Total number:', landsat_AOI.size().getInfo())

least_cloudy = ee.Image(landsat_AOI.sort('CLOUD_COVER').first())
date = ee.Date(least_cloudy.get('system:time_start'))
time = date.getInfo()['value']/1000.
dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

ndvi = least_cloudy.normalizedDifference(['B5', 'B4'])
palette = ['red', 'yellow', 'green']
ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 1024,
                   'palette': palette,
                   'region': phoenix_AOI}

phoenix_map = folium.Map(location=[33.4484, -112.0740], zoom_start=7)
phoenix_map.addLayer(ndvi, ndvi_parameters, name = "NDVI")
folium.LayerControl().add_to(phoenix_map)

phoenix_map


Total number: 7


test
